In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
import pandas as pd
import numpy as np

In [2]:
windowfy(10, True)

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%
Oversampling train users

Finished windowfying


In [6]:
# methods

def windowfy_sliding_training(users, window_size, pos_param_range_max=-1, neg_param_range_max=-1):
    users_windows = []
    for user, user_writings in users.items():     
        if user_writings[0]["g_truth"] == 1:
            param_range_max = pos_param_range_max
        else:
            param_range_max = neg_param_range_max
        count = 0
        if param_range_max < 0 or param_range_max > len(user_writings):
            range_max = len(user_writings)
            writings = user_writings.copy()
        else:
            range_max = param_range_max
            writings = user_writings.copy()[:range_max]
        for i in range(0, range_max):  # TODO parametrizar esto
            #if i < window_size and len(writings) > (i + 1):
            #    window = writings[:i + 1]  # rellenamos mientras "no nos llegan los demas mensajes"
            if len(writings) < (i + window_size):
                window = writings[i:range_max]  # TODO comprobar este range_max
                continue
            else:
                window = writings[i:i + window_size]

            if len(window) == 0:
                pass
                #print("Window: {}, i: {}, len(writings): {}".format(window, i, len(writings)))

            joined_window = join_window_elements(window)
            users_windows.append(joined_window)
            count += 1

    logger("Length of train data after windowfying: {}".format(len(users_windows)))
    return users_windows


def windowfy_sliding_testing(users, window_size, param_range_max=-1):
    users_windows = []
    for user, writings in users.items():
        count = 0
        if param_range_max < 0 or param_range_max > len(writings):
            range_max = len(writings)
        else:
            range_max = param_range_max
        for i in range(0, range_max):  # TODO parametrizar esto
            if i < window_size and len(writings) > (i+1):
                window = writings[:i+1] # rellenamos mientras "no nos llegan los demas mensajes" # todo cambiar esto
            elif len(writings) < (i + window_size):
                #window = writings[i:range_max]  # TODO comprobar este range_max
                window = []
            else:
                window = writings[i:i + window_size]

            if len(window) == 0:
                pass
                #print("Window: {}, i: {}, len(writings): {}".format(window, i, len(writings)))
            else:
                joined_window = join_window_elements(window)
                users_windows.append(joined_window)
                count += 1

    return users_windows


def join_all_elements(users):
    joined_writings = []
    for user, writings in users.items():
        joined_writings.append(join_window_elements(writings))

    return joined_writings


def join_window_elements(window: list) -> dict:
    joint_window = {}
    flatten = lambda l: [item for sublist in l for item in sublist]

    for key in window[0].keys():
        key_list = [message[key] for message in window]
        if type(key_list[0]) is list:
            joint_window[key] = flatten(key_list)
        elif key == 'user':
            joint_window[key] = key_list[0]
        elif key == 'g_truth':
            joint_window[key] = key_list[0]
        elif key == 'date':
            joint_window[key] = key_list
        elif key == 'sequence':
            joint_window[key] = key_list[-1]
        else:
            joint_window[key] = ' .'.join(key_list)

    return joint_window

In [7]:
train_window = windowfy_sliding_training(train_users, 10)
test_window = windowfy_sliding_testing(test_users, 10)

Length of train data after windowfying: 167374


In [8]:
train_window_frame = pd.DataFrame(train_window)
test_window_frame = pd.DataFrame(test_window)

In [12]:
train_window_frame.keys

<bound method NDFrame.keys of                                                      text         user  \
0       I'm a little late for this, but I agree. I tho...     subject0   
1       Regular Monster was ruined for me. I was drink...     subject0   
2       Can you elaborate on the prostitution? .Most f...     subject0   
3       Most free porn sites (Pornhub, Redtube) are ow...     subject0   
4       Why? You can not use the site if you feel "exp...     subject0   
...                                                   ...          ...   
167369  u/sayswut .I wasn't even subscribed, but got b...  subject9961   
167370  I wasn't even subscribed, but got banned.. I j...  subject9961   
167371  You ain't fooling anyone you muppet .banana sl...  subject9961   
167372  banana slugs  lt; Other UC Mascots .Nice .Can ...  subject9961   
167373  Nice .Can confirm. At Merced .Alexa play Despa...  subject9961   

        g_truth                                         clean_text  \
0          

In [13]:
X_train = train_window_frame["clean_text"] 
X_test = test_window_frame["clean_text"]
y_train = np.array(train_window_frame["g_truth"])
y_test = np.array(test_window_frame["g_truth"])

In [14]:
save_pickle("pickles", "X_train.pkl", X_train)
save_pickle("pickles", "X_test.pkl", X_test)
save_pickle("pickles", "y_train.pkl", y_train)
save_pickle("pickles", "y_test.pkl", y_test)
save_pickle("pickles", "train_window.pkl", train_window_frame)
save_pickle("pickles", "test_window.pkl", test_window_frame)